<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/05wk-2
    .ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [1]:
#{{<video https://youtu.be/playlist?list=PLQqh36zP38-z7s7FppJtYXFUnJzw88qWg&si=3TuaJ7IiiIG6QT7X>}}

# 2. Imports

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. MNIST

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True,transform=torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True,transform=torchvision.transforms.ToTensor())
X,y = next(iter(torch.utils.data.DataLoader(train_dataset,batch_size=6000,shuffle=True)))
XX,yy = next(iter(torch.utils.data.DataLoader(train_dataset,batch_size=1000,shuffle=True)))

In [26]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,32,kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # (n,32,?,?)
    torch.nn.Conv2d(32,32,kernel_size=3),
    torch.nn.ReLU(),
    torch.nn.Flatten(), # (n,???)
    torch.nn.Linear(3200,10)
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
X = X.to("cuda:0")
XX = XX.to("cuda:0")
y = y.to("cuda:0")
yy = yy.to("cuda:0")
net.to("cuda:0")
#---#
for epoc in range(100):
    #1
    netout = net(X)
    #2
    loss = loss_fn(netout,y)
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [33]:
(net(X).argmax(axis=1) == y).float().mean()

tensor(0.9862, device='cuda:0')

In [34]:
(net(XX).argmax(axis=1) == yy).float().mean()

tensor(0.9730, device='cuda:0')

In [36]:
torch.cuda.empty_cache()

# 4. CIFAR10

In [39]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,transform=torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,transform=torchvision.transforms.ToTensor())
X,y = next(iter(torch.utils.data.DataLoader(train_dataset,batch_size=10000,shuffle=True)))
XX,yy = next(iter(torch.utils.data.DataLoader(train_dataset,batch_size=2000,shuffle=True)))

## A. 직접설계 

In [53]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(3,32,kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # (n,32,?,?)
    torch.nn.Conv2d(32,32,kernel_size=3),
    torch.nn.ReLU(),
    torch.nn.Flatten(), # (n,???)
    torch.nn.Linear(4608,10)
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())
#---#
X = X.to("cuda:0")
XX = XX.to("cuda:0")
y = y.to("cuda:0")
yy = yy.to("cuda:0")
net.to("cuda:0")
#---#
for epoc in range(100):
    #1
    netout = net(X)
    #2
    loss = loss_fn(netout,y)
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [57]:
(net(X).argmax(axis=1) == y).float().mean()

tensor(0.5361, device='cuda:0')

In [58]:
(net(XX).argmax(axis=1) == yy).float().mean()

tensor(0.4825, device='cuda:0')

In [59]:
torch.cuda.empty_cache()

## B. 알렉스넷? 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Comparison_image_neural_networks.svg/960px-Comparison_image_neural_networks.svg.png)

In [64]:
img = torch.randn([1,3,224,224])

In [65]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(3,96,kernel_size=(11,11),stride=4),
    torch.nn.ReLU(),    
    torch.nn.MaxPool2d((3,3),stride=2), # default stride는 3
    torch.nn.Conv2d(96,256,kernel_size=(5,5),padding=2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((3,3),stride=2), # default stride는 3
    torch.nn.Conv2d(256,384,kernel_size=(3,3),padding=1),
    torch.nn.ReLU(),
    torch.nn.Conv2d(384,384,kernel_size=(3,3),padding=1),
    torch.nn.ReLU(),    
    torch.nn.Conv2d(384,256,kernel_size=(3,3),padding=1),
    torch.nn.ReLU(),    
    torch.nn.MaxPool2d((3,3),stride=2),
    torch.nn.Flatten(),
    torch.nn.Linear(6400,4096),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(4096,4096),        
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),    
    torch.nn.Linear(4096,1000),
)

In [71]:
net[:3](img).shape

torch.Size([1, 96, 26, 26])

## C. 알렉스넷으로 ImageNet 적합

In [74]:
net[-1] = torch.nn.Linear(4096,10)

In [80]:
img = torch.randn(1,3,32,32)

In [81]:
net(img)

RuntimeError: Given input size: (256x2x2). Calculated output size: (256x0x0). Output size is too small

In [90]:
net[:5](img).shape

torch.Size([1, 256, 2, 2])

In [92]:
net[5](net[:5](img))

RuntimeError: Given input size: (256x2x2). Calculated output size: (256x0x0). Output size is too small

## D. renset18

`-` res: <https://arxiv.org/pdf/1512.03385>

In [102]:
net = torchvision.models.resnet18()
net.fc = torch.nn.Linear(512,10)

## E. resnet18, pretrained=True

# 5. XAI 란? 

<https://brunch.co.kr/@hvnpoet/140>